In [ ]:
import pandas as pd

df = pd.read_excel("data/ver2_deduped_fix.xlsx")
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,spotify_id,daily_rank,popularity,is_explicit,duration_ms,album_name,album_release_date,...,overall_sentiments,overall_emotions,overall_topics,popularity_new,is_collab,release_date_update,lyrics_sections,key_translated,mode_numeric,popularity_tier
0,5870,55150,22531,2GxrNKugF82CnoRFbQfzPf,9.0,100.0,0.0,142514.0,i like the way you kiss me,2024-03-19,...,positive sentiment,emotion is sadness,song about loss and grief,83,0,2024-03-19,['I like the way you kiss me\nI like the way y...,B,1,High
1,1445,14661,6094,3qhlB30KknSejmIvZZLjOD,14.0,100.0,0.0,159245.0,DECIDE,2022-09-16,...,negative sentiment,emotion is sadness,song about loss and grief,89,0,2022-09-16,"['Just one more tear to cry, one teardrop from...",D,1,High
2,1070,11150,4323,1BxfuPKGuaTgP7aM0Bbdwr,5.0,100.0,0.0,178426.0,Lover,2019-08-23,...,negative sentiment,emotion is sadness,song about loss and grief,88,0,2019-08-23,"['(\nYeah, yeah, yeah, yeah\n)', ""Fever dream ...",A,1,High
3,5853,55133,22484,3rUGC1vUpkDG9CZFHMur1t,6.0,100.0,1.0,131872.0,greedy,2023-09-15,...,negative sentiment,emotion is optimism,song about ambition and success,87,0,2023-09-15,"['(\nWoo\n)', 'He said, ""Are you serious? I\'v...",F#,0,High
4,352,3712,2008,6dOtVTDdiauQNBQEDOtlAB,2.0,100.0,0.0,210373.0,HIT ME HARD AND SOFT,2024-05-17,...,negative sentiment,emotion is sadness,song about love,98,0,2024-05-17,"['(But I wanna stay)', ""I want you to stay\n'T...",D,1,High


In [ ]:
len(df)

6003

In [ ]:
# Step 0: Categorize popularity into tiers
def categorize_popularity(score):
    if score >= 60:
        return 'High'
    elif score >= 30:
        return 'Medium'
    else:
        return 'Low'

df['popularity_tier'] = df['popularity'].apply(categorize_popularity)

df.to_excel('data/ver3.xlsx', index=False)

# Step 1: One-hot encode the emotion column
emotion_dummies = pd.get_dummies(df['overall_emotions'])

# Step 2: Combine with the original dataframe
df_encoded = pd.concat([df, emotion_dummies], axis=1)

# Step 3: Define only the columns you want to analyze
numerical_features = ['danceability', 'valence', 'energy', 'acousticness',
                      'instrumentalness', 'liveness', 'speechiness', 'loudness']
emotion_columns = emotion_dummies.columns.tolist()
columns_of_interest = numerical_features + emotion_columns

# Optional: Create a function to compute correlation matrix for a given tier
def compute_correlation_by_tier(tier):
    subset = df_encoded[df_encoded['popularity_tier'] == tier]
    subset_df = subset[columns_of_interest]
    return subset_df.corr().loc[numerical_features, emotion_columns]

# Step 4: Compute correlation matrices by tier
corr_high = compute_correlation_by_tier('High')
corr_medium = compute_correlation_by_tier('Medium')
corr_low = compute_correlation_by_tier('Low')

# Helper function to tidy and tag the correlation matrix
def tidy_corr_matrix(corr_matrix, tier_label):
    df_tidy = corr_matrix.reset_index().melt(id_vars='index')
    df_tidy.columns = ['musical_feature', 'emotion', 'correlation']
    df_tidy['popularity_tier'] = tier_label
    return df_tidy

# Compute tidy correlations for each tier
tidy_high = tidy_corr_matrix(corr_high, 'High')
tidy_medium = tidy_corr_matrix(corr_medium, 'Medium')
tidy_low = tidy_corr_matrix(corr_low, 'Low')

# Combine into a single DataFrame
combined_corr_df = pd.concat([tidy_high, tidy_medium, tidy_low], ignore_index=True)

combined_corr_df.to_excel('data/emotion_music_correlation_by_popularity.xlsx', index=False)

In [ ]:
# Group by emotion and popularity tier
group_cols = ['overall_emotions', 'popularity_tier']

# Raw count
raw_counts = df.groupby(group_cols)['key_translated'].value_counts().unstack().fillna(0)

# Proportions within each emotion-tier
proportions = df.groupby(group_cols)['key_translated'].value_counts(normalize=True).unstack().fillna(0)

# Reset indexes
raw_counts = raw_counts.reset_index()
proportions = proportions.reset_index()

# Melt to long format
counts_melted = raw_counts.melt(id_vars=group_cols,
                                var_name='key_translated',
                                value_name='count')

proportions_melted = proportions.melt(id_vars=group_cols,
                                      var_name='key_translated',
                                      value_name='proportion')

# Merge counts and proportions
tableau_ready_df = pd.merge(proportions_melted, counts_melted,
                            on=group_cols + ['key_translated'])

# # Rank keys within each emotion-tier group by proportion (descending)
# tableau_ready_df['rank'] = tableau_ready_df.groupby(group_cols)['proportion']\
#                                            .rank(method='dense', ascending=False).astype(int)

# Save to Excel
tableau_ready_df.to_excel("data/emotion_key_proportions.xlsx", index=False)
